<a href="https://colab.research.google.com/github/zakonreal/ds_homework/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22PretrainedBERT_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_pretrained_bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
# transformers
# huggingface

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
phrase = 'He ran quickly after the red bus and caught it'
tokenized_phrase = tokenizer.tokenize(phrase)
tokenized_phrase = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(phrase))
bert_model = BertModel.from_pretrained('bert-base-multilingual-uncased').eval()

with torch.no_grad():
    embeddings = bert_model(torch.tensor(tokenized_phrase).unsqueeze(0))


In [ ]:
# embeddings.size

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
phrase = 'He ran quickly after the red bus and caught it'
tokenized_phrase = tokenizer.tokenize(phrase)

In [ ]:
tokenized_phrase

['he', 'ran', 'quickly', 'after', 'the', 'red', 'bus', 'and', 'caught', 'it']

In [ ]:
bert_model = BertModel.from_pretrained('bert-base-multilingual-uncased').eval()

In [ ]:
tokenized_phrase = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(phrase))

In [ ]:
tokenized_phrase

[10191, 15695, 23559, 10515, 10103, 10452, 15952, 10110, 34576, 10197]

In [ ]:
with torch.no_grad():
    embeddings = bert_model(torch.tensor(tokenized_phrase).unsqueeze(0))


In [ ]:
embeddings[0][0].size()

torch.Size([1, 10, 768])

In [ ]:
classifier = nn.Linear(768, 20)

predict = classifier(embeddings[0][0])

classifier


Linear(in_features=768, out_features=20, bias=True)

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

class DatasetSeq(Dataset):
    def __init__(self, data_dir, train_lang='en'):
	#open file
        tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
        bert_model = BertModel.from_pretrained('bert-base-multilingual-uncased').eval()
        with open(data_dir + train_lang + '.train', 'r') as f:
            train = f.read().split('\n\n')

        # delete extra tag markup
        train = [x for x in train if not '_ ' in x]
	    #init vocabs of tokens for encoding {<str> token: <int> id}
        self.target_vocab = {} # {p: 1, a: 2, r: 3, pu: 4}
        self.word_vocab = {} # {cat: 1, sat: 2, on: 3, mat: 4, '.': 5}
	    
        # Cat sat on mat. -> [1, 2, 3, 4, 5]
        # p    a  r  p pu -> [1, 2, 3, 1, 4]
        # chars  -> [1, 2, 3, 4, 5, 2, 3, 4]

	    #init encoded sequences lists (processed data)
        self.embeddings = []
        self.encoded_targets = []
        # n=1 because first value is padding
        n_word = 1
        n_target = 1
        for line in train:
            sequence = []
            target = []
            for item in line.split('\n'):
                if item != '':
                    word, label = item.split(' ')

                    if self.word_vocab.get(word) is None:
                        self.word_vocab[word] = n_word
                        n_word += 1
                    if self.target_vocab.get(label) is None:
                        self.target_vocab[label] = n_target
                        n_target += 1
                    
                    sequence.append(word)
                    target.append(self.target_vocab[label])
            sequence = ' '.join(sequence)
            
            tokens = tokenizer.tokenize(sequence)
            tokenized = tokenizer.convert_tokens_to_ids(tokens)
            with torch.no_grad():
                embeddings = bert_model(torch.tensor(tokenized).unsqueeze(0))
            #TODO check tokens with ## f.e. quckly -> quik## ##ly -> emb[quick##] + emb[##ly]
            # impossible -> im## ##possible
            embeddings = embeddings[0][0]
            embeddings_processed = []
            if len(tokens) != embeddings.size(1):
                print('--', sequence, tokens)
            for i, token in enumerate(tokens):
                if token.startswith('##') and i >= 1:
                    embeddings_processed[-1] = embeddings_processed[-1] + embeddings[0, i, :].unsqueeze(0)
                elif token.startswith('##') and i == 0:
                    print(sequence)
                else:
                    embeddings_processed.append(embeddings[0, i, :].unsqueeze(0))

            self.embeddings.append(torch.cat(embeddings_processed, dim=1))
            self.encoded_targets.append(target)

    def __len__(self):
        return len(self.embeddings) #было len(self.encoded_sequences) - выдавалась ошибка, что нет такого атрибута, изменил

    def __getitem__(self, index):
        return {
            'data': self.embeddings[index], # [1, 2, 3, 4, 6] len=5
            'target': self.encoded_targets[index], #  (1)
        }

In [ ]:
def collate_fn(batch):
    data = []
    target = []
    for item in batch:
        data.append(item['data'])
        target.append(torch.as_tensor(item['target']))
    data = pad_sequence(data, batch_first=True, padding_value=0.)
    target = pad_sequence(target, batch_first=True, padding_value=0)

    return {'data': data, 'target': target}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = 'drive/My Drive/'
train_lang = 'en'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = DatasetSeq(data_dir)

In [ ]:
%%time
from torch.utils.data import DataLoader
import datetime

n_classes = len(dataset.target_vocab) + 1
n_epochs = 10
cuda_device = 0
batch_size = 100
device = f'cuda:{cuda_device}' if cuda_device != -1 else 'cpu'
optim = torch.optim.Adam(classifier.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()
start1 = datetime.datetime.now()
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = classifier(batch['data'].to(device))
        loss = loss_func(predict.view(-1, n_classes),
                         batch['target'].to(device).view(-1), 
                         )
        loss.backward()
        optim.step()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
   
    torch.save(classifier.state_dict(), f'./rnn_chkpt_{epoch}.pth')
train_time = datetime.datetime.now() - start1
train_loss = loss.item()
print(train_time)
print(train_loss)

RuntimeError: ignored

In [ ]:
#example
import datetime
phrase = 'He ran quickly after the red bus and caught it'
words = phrase.split(' ')
tokens = [dataset.word_vocab[w] for w in words]

start = datetime.datetime.now()
with torch.no_grad():
    classifier.eval()
    predict = classifier(torch.tensor(tokens).unsqueeze(0).to(device)) # 1 x T x N_classes
    labels = torch.argmax(predict, dim=-1).squeeze().cpu().detach().tolist()
    inference_time = datetime.datetime.now() - start

target_labels = list(dataset.target_vocab.keys())
print([target_labels[l-1] for l in labels])
print(inference_time)

RuntimeError: ignored